In [1]:
#-------------------------------------------------------------------------------
# Los datos de Datawarehouse(DWH) se bajaron del Google Cloud Platform a DRIVE
# se preparo la data para ser leida por el modelo y solo contiene los 
# registros MGN(Migracion Neta)
# Se monta el DRIVE para leer la data
#-------------------------------------------------------------------------------
# OBJETIVO: Generar un archivo .csv en donde el modelo de machine learning 
# en base a la informacion del 2010 hasta el 2020, va a predecir el 
# indice de migracion neta del año 2021, usando el modelo de regresion lineal
#-------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#-------------------------------------------------------------------------------
# codigo machine learning modelo de regresion predictiva
# predecir año 2021 con el indice de migracion neta, tomando como data
# los años del 2010 al 2020 que estan en la web del banco mundial
#-------------------------------------------------------------------------------

# cargamos las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# hacemos la ingesta de la data de remesas por paises
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLMGNprepDATA.csv')


# Seleccionar país para la predicción
dfpaises = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/tablapaises.csv')
#paises= ['ARG', 'PER', 'COL']
paises = dfpaises['codigopais']
indice= 1
paispredecir = paises[indice]

# creamos un dataframe copia para añadir la nueva columna de prediccion
# y grabar la prediccion que haga el machine learning
df_ml = df.copy()
df_ml['prediccion']= 0

# Convertir de ancho a largo
df_melted = pd.melt(df, id_vars=['codigopais'], var_name='year', value_name='remittances')

# eliminamos valores NaN
df_melted.dropna(inplace=True)

# Convertir year a variable numérica
df_melted['year'] = pd.to_numeric(df_melted['year'])

# Convertir remittances a tipo float
#df_melted['remittances'] = df_melted['remittances'].str.replace(',', '.').astype(float)

# Convertir codigopais a variable numérica mediante LabelEncoder
le = LabelEncoder()
df_melted['codigopais'] = le.fit_transform(df_melted['codigopais'])



# hacemos un for por todos los paises
for indice in range(len(paises)):
  paispredecir = paises[indice]
   
  # Filtrar datos por país seleccionado
  df_pais = df_melted[df_melted['codigopais'] == le.transform([paispredecir])[0]]

  # Ajustar modelo de regresión
  X = df_pais[df_pais['year'] <= 2020][['year', 'codigopais']]
  y = df_pais[df_pais['year'] <= 2020]['remittances']
  reg = LinearRegression().fit(X, y)

  # Hacer predicciones para el país seleccionado
  añopredecir = 2021
  X_pred = np.array([[añopredecir, le.transform([paispredecir])[0]]])
  y_pred = reg.predict(X_pred)
  #print(f"Predicción para {paispredecir} en {añopredecir}: {y_pred[0]:.4f}")
  print(f"Predicción para {paispredecir} en {añopredecir}: {int(y_pred[0])}")

  df_ml.loc[df_ml['codigopais'] == paispredecir,'prediccion'] = int(y_pred[0])


# al finalizar el for, guarda el nuevo .csv del dataframe: df_ml
df_ml.to_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLMGNprepDATAPREDICCIONML.csv', index = False)
# 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

Predicción para ARG en 2021: 6594
Predicción para BOL en 2021: -6717
Predicción para BRA en 2021: 72841
Predicción para CAN en 2021: 352872
Predicción para CHL en 2021: 224446
Predicción para COL en 2021: 437524
Predicción para CRI en 2021: 3305
Predicción para CUB en 2021: -7005
Predicción para ECU en 2021: 92745
Predicción para SLV en 2021: -42165
Predicción para USA en 2021: 1076312
Predicción para GTM en 2021: -48218
Predicción para GUY en 2021: 2441
Predicción para HTI en 2021: -36354
Predicción para HND en 2021: -6905
Predicción para JAM en 2021: -6508
Predicción para MEX en 2021: -40183
Predicción para NIC en 2021: -14669
Predicción para PAN en 2021: 10771
Predicción para PRY en 2021: -11157
Predicción para PER en 2021: 247475
Predicción para DOM en 2021: -24331
Predicción para URY en 2021: -1025
Predicción para VEN en 2021: -1030626


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [10]:
df_comprobar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLMGNprepDATAPREDICCIONML.csv')


In [11]:
df_comprobar

,codigopais,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,prediccion
0,ATG,295,247,203,156,108,60,16,0,0,0,0,0
1,LCA,438,181,11,0,0,0,0,0,0,0,0,0
2,BLZ,1644,1583,1529,1504,1452,1363,1283,1222,1185,1164,0,0
3,BRB,-36,-54,-64,-65,-65,-65,-65,-65,-65,-65,0,0
4,BHS,1899,1384,1048,1001,1001,1001,1001,1001,1001,1001,0,0
5,GRD,-453,-292,-204,-190,-190,-190,-190,-190,-190,-190,0,0
6,JAM,-14244,-12578,-11494,-11329,-11329,-11329,-11329,-11329,-11329,-11329,0,-6508
7,DMA,-243,0,158,-445,1445,-393,179,187,194,326,100,0
8,GUY,-14327,-14512,-7152,-6912,-6715,-6524,-6311,-6110,30080,-24678,802,2441
9,PRY,-26173,-20435,-17761,-17238,-16990,-16833,-16734,-16628,-16470,-16270,-8230,-11157
